# Real-Time Experimental Results Visualizer

## Setup

In [ ]:
from code.excelmem.excelmemvis import main as excel_main
from code.libremem.librememvis import main as libre_main

import plotly.graph_objects as go
import multiprocessing as mp

In [2]:
fv_fig = go.FigureWidget(
    layout=go.Layout(
        title=go.layout.Title(text="Formula-Value Live Data"),
        xaxis=dict(title="Rows"),
        yaxis=dict(title="Formula Size (MB)")
    )
)
fv_fig.add_scatter(name='Formula-Value (MB)')
fv_fig

FigureWidget({
    'data': [{'name': 'Formula-Value (MB)', 'type': 'scatter', 'uid': 'bb3e616c-98d6-43c5-9139-…

In [3]:
vo_fig = go.FigureWidget(
    layout=go.Layout(
        title=go.layout.Title(text="Value-Only Live Data"),
        xaxis=dict(title="Rows"),
        yaxis=dict(title="Value Size (MB)")
    )
)
vo_fig.add_scatter(name='Value-Only (MB)')
vo_fig

FigureWidget({
    'data': [{'name': 'Value-Only (MB)', 'type': 'scatter', 'uid': '279a363d-76dc-41b1-9cbf-d1a…

In [4]:
barplt = go.FigureWidget(
    layout=go.Layout(
        title=go.layout.Title(text="Live Data"),
        xaxis=dict(title="Rows"),
        yaxis=dict(title="Size (MB)")
    )
)
barplt.add_bar(name='Value-Only (MB)')
barplt.add_bar(name='Formula-Value (MB)')
barplt

FigureWidget({
    'data': [{'name': 'Value-Only (MB)', 'type': 'bar', 'uid': 'b72047fc-649f-4b3e-881f-772e836…

## Place your own settings here

In [5]:
# Either "libre" or "excel"
APPLICATION = "excel"

SOFFICE_DIR = "C:/Program Files/LibreOffice/program/soffice"
INPUTS_PATH = os.path.join("input-data", "inputs")
FV_INPUTDIR = "formula-value"
VO_INPUTDIR = "value-only"
OUTPUT_NAME = "results"
OUTDIR_NAME = "rcbs-5trials-rand-1col-run1"
INTEGER_ARG = 5

## Run the following cells for real-time results

In [6]:
parent_conn, child_conn = mp.Pipe()

if APPLICATION.lower() == "excel":
  process = mp.Process(target=excel_main
    , args=(child_conn, INPUTS_PATH,)
    , kwargs={
      "fv_inputdir"   : FV_INPUTDIR
      , "vo_inputdir" : VO_INPUTDIR
      , "output_path" : os.path.join(OUTPUT_NAME, "excel")
      , "outdir_name" : OUTDIR_NAME
      , "trials"      : INTEGER_ARG
    }
  )

elif APPLICATION.lower() == "libre":
  process = mp.Process(target=libre_main
    , args=(child_conn, INPUTS_PATH,)
    , kwargs={
      "soffice_path"  : SOFFICE_DIR
      , "fv_inputdir" : FV_INPUTDIR
      , "vo_inputdir" : VO_INPUTDIR
      , "output_path" : os.path.join(OUTPUT_NAME, "libre")
      , "outdir_name" : OUTDIR_NAME
      , "pollseconds" : INTEGER_ARG
    }
  )

else:
  process = None
  print(f"{APPLICATION} is an invalid option.")

In [7]:
if process is not None:
    process.start()
    while True:
        item = parent_conn.recv()
        if type(item) == dict:
            vo_rowsizes = []; vo_memsizes = []; fv_rowsizes = []; fv_memsizes = []
            for r, d in sorted(item.items(), key=lambda pair: pair[0]):
                if "Value USS (MB)"   in d: vo_rowsizes.append(str(r)); vo_memsizes.append(d["Value USS (MB)"  ])
                if "Formula USS (MB)" in d: fv_rowsizes.append(str(r)); fv_memsizes.append(d["Formula USS (MB)"])
            barplt.data[0].x = vo_fig.data[0].x = vo_rowsizes
            barplt.data[1].x = fv_fig.data[0].x = fv_rowsizes
            barplt.data[0].y = vo_fig.data[0].y = vo_memsizes
            barplt.data[1].y = fv_fig.data[0].y = fv_memsizes
        if type(item) == str:   print(item)
        if item is None:        break
    process.join()
    process.terminate()

Closing all running instances of EXCEL.EXE (if any)
Opening vo-700000.xlsx (trial 1)
Opening vo-700000.xlsx (trial 2)
Opening vo-700000.xlsx (trial 3)
Opening vo-700000.xlsx (trial 4)
Opening vo-700000.xlsx (trial 5)
{'Value Peak WSS (MB)': 187.80569600000004, 'Value WSS (MB)': 187.26775466666663, 'Value RSS (MB)': 187.26912000000002, 'Value USS (MB)': 102.4150186666667}
Opening vo-20000.xlsx (trial 1)
Opening vo-20000.xlsx (trial 2)
Opening vo-20000.xlsx (trial 3)
Opening vo-20000.xlsx (trial 4)
Opening vo-20000.xlsx (trial 5)
{'Value Peak WSS (MB)': 140.27298133333338, 'Value WSS (MB)': 140.26478933333334, 'Value RSS (MB)': 140.26478933333334, 'Value USS (MB)': 61.36490666666666}
Opening vo-300000.xlsx (trial 1)
Opening vo-300000.xlsx (trial 2)
Opening vo-300000.xlsx (trial 3)
Opening vo-300000.xlsx (trial 4)
Opening vo-300000.xlsx (trial 5)
{'Value Peak WSS (MB)': 158.36364800000004, 'Value WSS (MB)': 158.28991999999997, 'Value RSS (MB)': 158.28991999999997, 'Value USS (MB)': 77.579

In [8]:
# Timing notes

# EXCEL:
# large-RCBS-1col
#   Up to 1 million: Total time (HH:MM:SS): 06:56:18
# RCBS-1col
#   Up to 1 million: Total time (HH:MM:SS): 07:05:28
# RLS-1col
#   Up to 1 million: Total time (HH:MM:SS): 00:06:18
# RLS-5col
#   Up to 1 million: Total time (HH:MM:SS): 00:21:21